# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should consider factors like size, location, amenities, and cost.

Action: Search the internet
Action Input: {"search_query": "tech event venues in San Francisco"} 


Search results: Title: The 16 Best Networking Venues for Rent in San Francisco, CA
Link: https://www.tagvenue.com/us/hire/networking-venues/san-francisco
Snippet: Known for its tech and startup scene, SoMa provides modern and creative spaces ideal for tech networking events. Attractions like the San Francisco Museum ...
---
Title: 36 Best Networking Event Venues In San Francisco, CA - Peerspace
Link: https://www.peerspace.com/venues/san-francisco--ca/networking-venue
Snippet: Easily rent a networking and job fair venue in San 

This is the agent final answer: Venue: Radian Gallery
Location: South Beach, San Francisco, CA
Description: The Radian Gallery offers a modern and artistic setting, perfect for a tech innovation conference. The venue can accommodate 60 to 85 guests and is ideal for a panel event. It has a mix of professional and soulful ambiance with beautiful art throughout the space. The rental fee is $150 per hour.

Venue: Novela
Location: Financial District, San Francisco, CA
Description: Novela is a unique venue that pays homage to classic novels, authors, and characters. It offers "cocktails with character" and can accommodate 50 to 95 guests. The venue is ideal for a session at a tech innovation conference, with a hire fee of $750.

Venue: NocNoc
Location: Hayes Valley, San Francisco, CA
Description: NocNoc is an eccentric bar perfect for social gatherings. It can accommodate up to 95 people standing or 50 seated. The venue has a minimum spend of $175 per session and is a great space for network

Final Answer: The Radian Gallery, located in South Beach, San Francisco, CA, has a capacity of 85 participants and is available for booking at $150 per hour. Since the event requires space for 500 participants, I have found a list of large event venues in San Francisco that may meet the capacity requirements. Here are a few options to consider:
1. The San Francisco Peninsula - Unique Venues
2. Top 20 Venues in San Francisco, CA - PartySlate
3. Unique San Francisco Venues That Should Be On Every Meeting Planners Radar

I recommend exploring these options further to secure a venue that best fits the event's needs.

> Finished chain.
 

The Radian Gallery, located in South Beach, San Francisco, CA, has a capacity of 85 participants and is available for booking at $150 per hour. Since the event requires space for 500 participants, I have found a list of large event venues in San Francisco that may meet the capacity requirements. Here are a few options to consider:
1. The San Francisco Peni

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: 
I have gathered useful information on effective marketing activities and attendee engagement for the Tech Innovation Conference.

Final Answer: 
# Marketing Activities and Attendee Engagement Report

## Marketing Activities:
1. Determine key messaging for the Tech Innovation Conference to attract potential attendees.
2. Identify experts in the tech innovation field to be part of the conference committee.
3. Find influencers in the industry to help spread the word about the conference.
4. Optimize the conference website with relevant information and registration details.
5. Create engaging content such as teaser videos to promote the event.
6. Utilize social media platforms to promote the conference and engage with the audience.
7. Develop a strong brand presence for the conference to attract attendees.
8. Offer incentives to potential attendees to encourage participation.
9. Utilize speaker and sponsor endorsements to increase credibility and interest in the conference.
10. U

- Display the generated `venue_details.json` file.

In [15]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'South Beach, San Francisco, CA',
 'booking_status': '$150 per hour',
 'capacity': 85,
 'name': 'Radian Gallery'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [16]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities and Attendee Engagement Report

## Marketing Activities:
1. Determine key messaging for the Tech Innovation Conference to attract potential attendees.
2. Identify experts in the tech innovation field to be part of the conference committee.
3. Find influencers in the industry to help spread the word about the conference.
4. Optimize the conference website with relevant information and registration details.
5. Create engaging content such as teaser videos to promote the event.
6. Utilize social media platforms to promote the conference and engage with the audience.
7. Develop a strong brand presence for the conference to attract attendees.
8. Offer incentives to potential attendees to encourage participation.
9. Utilize speaker and sponsor endorsements to increase credibility and interest in the conference.
10. Use video marketing to showcase the conference highlights and benefits.
11. Implement a hashtag campaign to create buzz and encourage sharing of conference content.
12. Conduct online polls to gather feedback and engage with potential attendees.

## Attendee Engagement Ideas:
1. Utilize session Q&A's to understand attendees' interests and concerns.
2. Offer live chat options during virtual sessions for real-time engagement.
3. Conduct pre-event surveys and polls to gather feedback and customize the conference experience.
4. Personalize invitations and communication to make attendees feel valued.
5. Use social media engagement and event hashtags to create a sense of community.
6. Encourage the use of cameras and microphones for interactive discussions.
7. Implement icebreakers and interactive activities to engage participants.
8. Provide speaker orientations to prepare attendees for the conference sessions.
9. Schedule entertainment and fun events to keep attendees engaged.
10. Check in regularly with attendees to address any concerns or feedback.

By implementing these marketing activities and attendee engagement ideas, the Tech Innovation Conference can effectively promote the event and maximize attendee participation.